In [113]:
# IPython Imports
# Used to make notebook wider, comment out for normal notebook
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
from IPython.display import clear_output

In [75]:
# Import Libraries
import boto3
import csv
import numpy as np
import pandas as pd
from pandas_profiling import ProfileReport
import matplotlib.pyplot as plt
import re
from typing import List, Dict, Callable

In [76]:
# GLOBAL HELPER FUNCTIONS
def print_ref(d_obj:Dict[str, any], row_len:int=4) -> None:
    ref = \
f""" ---------------------------
| OBJECT KEYS FOR REFERENCE |
 ---------------------------
"""
    row = 0
    for key in d_obj.keys():
        if row == row_len: 
            row = 0
            ref += "\n"
        ref += f" {key} |"
        row += 1
    print(ref)
    
def parse_col(col_list:List[str], data_frame:pd.DataFrame=None) -> List[str]:
    res, len_ = [], len(col_list) + 1
    df_exists = data_frame is not None
    for num, col in enumerate(col_list):
        example = data_frame[col].iloc[0] if df_exists else "None"
        ans = input(f"""
Item #: {num + 1}/{len_}
Column: {col}
Example: {example}
[y/n] Default[n] -> """).lower()
        clear_output()
        if ans == "y":
            res.append(col)
    return res


In [77]:
# GLOBAL VARS
BKT_NAME = 'ds-data-2020'

# DATASET NAMES
the_cc = "thecarconnectiondataset.csv" # Main Dataset
motortrend = "motortrend1974.csv" # Currently Not Using
car_feat_msrp = "carfeaturesmsrp.csv" # Currently Not Using

# SET DATA PATH/KEYS
KEY = the_cc

In [78]:
# Create boto3 s3 Client Object
s3 = boto3.client('s3')
# Create s3 Bucket Object -> Dict
obj = s3.get_object(Bucket=BKT_NAME, Key=KEY)

# obj_keys = obj.keys()
print_ref(obj)


 ---------------------------
| OBJECT KEYS FOR REFERENCE |
 ---------------------------
 ResponseMetadata | AcceptRanges | LastModified | ContentLength |
 ETag | ContentType | Metadata | Body |


In [79]:
# Might have to chunk data
chk_size = 2000


# Convert Bucket File to transposed pd.DataFrame
auto_df = pd.read_csv(obj['Body'], index_col=0, header=None, low_memory=False).T

In [80]:
# Rename known NaN columns
auto_df.rename(columns={ auto_df.columns[0]: "Model" }, inplace = True)

In [81]:
# Get only wanted columns
# col = parse_col(auto_df.columns, auto_df)

# col = ["Model", 'MSRP', 'Gas Mileage', 'Engine', 'EPA Class', 'Style Name', 'Drivetrain', 'Passenger Capacity', 'Passenger Doors', 'Body Style', 'Transmission', 'Base Curb Weight (lbs)', 'Wheelbase (in)', 'Min Ground Clearance (in)', 'Track Width, Front (in)', 'Track Width, Rear (in)', 'Height, Overall (in)', 'Fuel Economy Est-Combined (MPG)', 'SAE Net Torque @ RPM', 'Fuel System', 'Engine Type', 'SAE Net Horsepower @ RPM', 'Displacement', 'First Gear Ratio (:1)', 'Sixth Gear Ratio (:1)', 'Fourth Gear Ratio (:1)', 'Seventh Gear Ratio (:1)', 'Second Gear Ratio (:1)', 'Reverse Ratio (:1)', 'Fifth Gear Ratio (:1)', 'Eighth Gear Ratio (:1)', 'Third Gear Ratio (:1)', 'Final Drive Axle Ratio (:1)', 'Steering Type', 'Front Tire Size', 'Rear Tire Size']

In [82]:
# Save wanted col to txt file
# with open("data/wanted_col.txt", "w") as text_file:
#     for item in col:
#         text_file.write(item + "\n")

In [83]:
# Get list of column from wanted_col.txt
wanted_col = open("data/wanted_col.txt", "r").read().splitlines()
# Extract columns to new df
auto_df_clean = auto_df[wanted_col].copy()

In [84]:
# Clean Model Column to only contain year and name
auto_df_clean["Model"] =  [i.partition('Specs')[0] for i in auto_df_clean.Model]

In [85]:
# Create Year Column from Model Column
auto_df_clean.insert(0, "Year", [re.search("([^\s]+)",x).group() for x in auto_df_clean["Model"]])

In [86]:
# Cast Year Column to Int
auto_df_clean["Year"] = auto_df_clean["Year"].astype(int) 

In [87]:
# Clean Model Column to only contain name
auto_df_clean["Model"] =  [(re.search("\s(.*)", i).group()) for i in auto_df_clean.Model]

In [88]:
# Create Brand Column from Model Column
auto_df_clean.insert(1, "Brand", [re.search("([^\s]+)", x).group() for x in auto_df_clean["Model"]])

In [89]:
# Clean Model Column to only contain model
auto_df_clean["Model"] =  [(re.search("\s(.*)", i).group(1)) for i in auto_df_clean.Model]

In [90]:
# Get rid of all nan values in Gas Mileage
nans = pd.notnull(auto_df_clean['Gas Mileage'])
auto_df_clean = auto_df_clean[nans]

In [91]:
# Split Gas Mileage into City and Hwy
auto_df_clean.insert(4, "Gas Mileage (City)", [(x.partition("/")[0].partition(" ")[0]) for x in auto_df_clean["Gas Mileage"]])
auto_df_clean.insert(5, "Gas Mileage (Hwy)", [(x.partition("/")[2].partition(" ")[0]) for x in auto_df_clean["Gas Mileage"]])

In [92]:
# Gas Mileage to INT
auto_df_clean['Gas Mileage (City)'] = pd.to_numeric(auto_df_clean['Gas Mileage (City)'])
auto_df_clean['Gas Mileage (Hwy)'] = pd.to_numeric(auto_df_clean['Gas Mileage (Hwy)'])

In [93]:
# Drop Gas Mileage Column after split
auto_df_clean.drop(columns=["Gas Mileage"], inplace=True)

In [94]:
# Clean Gear Ratio Columns
gears = ["First Gear Ratio (:1)", "Second Gear Ratio (:1)", "Third Gear Ratio (:1)", "Fourth Gear Ratio (:1)", "Fifth Gear Ratio (:1)", "Sixth Gear Ratio (:1)", "Seventh Gear Ratio (:1)", "Eighth Gear Ratio (:1)", "Reverse Ratio (:1)", "Final Drive Axle Ratio (:1)"]

# Reorder Gears Numerically
for idx, column in enumerate(gears, 26):
    col = auto_df_clean[column]
    auto_df_clean.drop(labels=[column], axis=1,inplace = True)
    auto_df_clean.insert(idx, column, col)
    
# Cast NaN to values in col with "- TBD -"
for gear in gears:
    auto_df_clean.loc[auto_df_clean[gear].str.contains("TBD", na=False), gear] = np.nan

In [95]:
# Cast float to values in col with string arithmetic
"Notes: Some gear columns need to be sep with (,), (/), or (" ") delimiters"
for gear in gears:
    auto_df_clean[gear] = pd.to_numeric([
        (x.partition(",")[0]) if ((type(x) == str) and ("," in x)) 
        else (x.partition("/")[0]) if ((type(x) == str) and ("/" in x))
        else (x.partition("-")[0]) if ((type(x) == str) and ("-" in x))
        else (x.partition(" ")[0]) if ((type(x) == str) and (" " in x))
        else x for x in auto_df_clean[gear]
    ], downcast="float", errors="coerce")

In [96]:
# Cast MSRP to Numeric Float
auto_df_clean["MSRP"] = pd.to_numeric((auto_df_clean["MSRP"].replace('[\$,]', '', regex=True)), downcast="integer", errors="coerce")
# Rename MSRP to indicate $
auto_df_clean.rename(columns={ "MSRP": "MSRP($)" }, inplace = True)

In [97]:
# Get rid of all nan values in Horsepower/Torque
nans = pd.notnull(auto_df_clean['SAE Net Torque @ RPM'])
auto_df_clean = auto_df_clean[nans]

In [98]:
# Split SAE Horsepower into Horsepower and @ RPM
auto_df_clean.insert(4, "Horsepower", [(x.partition("@")[0]) for x in auto_df_clean["SAE Net Horsepower @ RPM"]])
auto_df_clean.insert(5, "@ RPM (HP)", [(x.partition("@")[2]) for x in auto_df_clean["SAE Net Horsepower @ RPM"]])

In [99]:
# Split SAE Torque into Torque and @ RPM
auto_df_clean.insert(6, "Torque", [(x.partition("@")[0]) for x in auto_df_clean["SAE Net Torque @ RPM"]])
auto_df_clean.insert(7, "@ RPM (TQ)", [(x.partition("@")[2]) for x in auto_df_clean["SAE Net Torque @ RPM"]])

In [100]:
# Clean and Cast Power Values (Horsepower / Torque)
pow_rpm_ratings = ["Horsepower", "Torque", "@ RPM (HP)", "@ RPM (TQ)"]
"Notes: Some Power columns need to be sep with (@), or (-) delimiters"
# Cast NaN to values in col with empty ratings or "- TBD -"
for col in pow_rpm_ratings:
    auto_df_clean.loc[auto_df_clean[col].str.contains("TBD", na=False), col] = np.nan
    if col in pow_rpm_ratings[:2]:
        auto_df_clean[col] = pd.to_numeric(auto_df_clean[col], downcast="float", errors="coerce")

In [101]:
# Drop SAE Horsepower and SAE Torque Columns after splits
auto_df_clean.drop(columns=["SAE Net Horsepower @ RPM"], inplace=True)
auto_df_clean.drop(columns=["SAE Net Torque @ RPM"], inplace=True)

In [119]:
# Clean Base Curb Weight and Cast to INT
auto_df_clean["Base Curb Weight (lbs)"] = pd.to_numeric(auto_df_clean["Base Curb Weight (lbs)"], downcast="integer", errors="coerce")

In [128]:
# Clean Displacement and Cast to INT
auto_df_clean["Displacement"] = pd.to_numeric(
    [(x.partition("L")[0]) if type(x) == str else x for x in auto_df_clean["Displacement"]], 
    downcast="float", 
    errors="coerce"
)
auto_df_clean.rename(columns={ "Displacement": "Displacement(L)" }, inplace = True)

In [130]:
# auto_df_clean.info()

In [132]:
# auto_df_clean.head()

In [134]:
# auto_df_clean.describe()

,Year,MSRP($),Horsepower,Torque,Gas Mileage (City),Gas Mileage (Hwy),Base Curb Weight (lbs),Displacement(L),First Gear Ratio (:1),Second Gear Ratio (:1),Third Gear Ratio (:1),Fourth Gear Ratio (:1),Fifth Gear Ratio (:1),Sixth Gear Ratio (:1),Seventh Gear Ratio (:1),Eighth Gear Ratio (:1),Reverse Ratio (:1),Final Drive Axle Ratio (:1)
count,26284.000000,26261.000000,26250.000000,26242.000000,26284.000000,26276.000000,18527.000000,25947.000000,24834.000000,23729.000000,23743.000000,23709.000000,19534.000000,13070.000000,4275.000000,3128.000000,24336.000000,18188.000000
mean,2010.382324,38138.281292,250.164688,257.343811,19.555014,26.283148,3607.858261,3.345928,3.727093,2.247718,1.494974,1.098211,0.908716,0.782912,0.784824,0.659625,3.147366,3.702522
std,6.054405,32625.769552,97.641388,102.690323,5.558793,6.245657,689.049250,1.283857,0.729955,0.517186,0.361223,0.300257,0.197890,0.153854,0.159695,0.064355,0.702880,0.658005
min,1996.000000,8599.000000,65.000000,65.000000,9.000000,11.000000,1808.000000,1.000000,0.400000,0.740000,0.450000,0.360000,0.080000,0.075000,0.000000,0.000000,0.730000,2.050000
25%,2006.000000,23280.000000,173.000000,177.000000,16.000000,21.000000,3160.000000,2.300000,3.060000,1.900000,1.296000,0.950000,0.760000,0.680000,0.730000,0.660000,2.700000,3.210000
50%,2011.000000,30240.000000,240.000000,253.000000,18.000000,26.000000,3499.000000,3.200000,3.750000,2.200000,1.490000,1.030000,0.860000,0.750000,0.839000,0.670000,3.290000,3.650000
75%,2015.000000,40965.000000,302.000000,305.000000,22.000000,30.000000,4045.000000,4.200000,4.210000,2.530000,1.720000,1.280000,1.000000,0.880000,0.840000,0.670000,3.530000,4.110000
max,2019.000000,548800.000000,808.000000,3350.000000,66.000000,70.000000,8591.000000,7.000000,15.950000,10.040000,6.360000,4.620000,3.830000,3.130000,4.330000,0.850000,14.070000,6.470000


In [105]:
high_hp_brands = auto_df_clean[["Year", "Brand", "Model", "Style Name", "Horsepower", "Torque"]][auto_df_clean["Horsepower"] > 600].groupby("Brand")

In [106]:
# auto_df_clean[auto_df_clean["Brand"] == "Porsche"]

In [107]:
# auto_df_clean["Engine Type"].unique()

In [108]:
# auto_df_clean["Brand"].unique()

|<br>|<br>|<br>|<br>|<br>|<br>|<br>|<br>|<br>|<br>|<br>|<br>|<br>


In [135]:
profile = ProfileReport(auto_df_clean, title='theCarConnection Profiling Report', html={'style':{'full_width':True}})

In [136]:
profile.to_widgets()